In [2]:
import numpy as np
import pandas as pd

In [3]:
sample_submission = pd.read_csv('sample_submission.csv')
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [4]:
print(train_data.shape)
print(test_data.shape)
train_data.head(5)
#test_data.head(5)
ids = test_data['id'].values
print(type(ids))

(26570, 26)
(20775, 25)
<class 'numpy.ndarray'>


In [10]:
#data preprocessing
X_pre = train_data.drop(['failure','id'], axis=1).to_numpy()
Y = train_data['failure'].values

from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
def preprossing(X_pre):
    enc = OrdinalEncoder()
    enc.fit(X_pre)
    X_pre = enc.transform(X_pre)

    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_mean.fit(X_pre)
    X_pre = imp_mean.transform(X_pre)

    '''scaler = MinMaxScaler()
    scaler.fit(X_pre)
    X_pre = scaler.transform(X_pre)'''
    return X_pre

from sklearn.model_selection import train_test_split
#split training and validation data
X = preprossing(X_pre)
train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size=0.2, shuffle=False)


val_y = val_y.reshape(val_y.shape[0], 1)
train_y = train_y.reshape(train_y.shape[0], 1)


In [11]:
#test data preprocessing
test_pre = test_data.drop('id', axis=1).to_numpy()

enc = OrdinalEncoder()
enc.fit(test_pre)
#print(enc.categories_)
test_pre = enc.transform(test_pre)

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(test_pre)
test_pre = imp_mean.transform(test_pre)

'''scaler = MinMaxScaler()
scaler.fit(test_pre)
test_pre = scaler.transform(test_pre)'''

test = test_pre


In [7]:
print(type(test))

<class 'numpy.ndarray'>


In [12]:
print(val_X)
print(train_X)

[[4.00000000e+00 6.32700000e+03 1.00000000e+00 ... 2.94600000e+03
  3.77900000e+03 1.95500000e+03]
 [4.00000000e+00 3.55100000e+03 1.00000000e+00 ... 2.52500000e+03
  2.80400000e+03 9.81000000e+03]
 [4.00000000e+00 3.11800000e+03 1.00000000e+00 ... 2.89100000e+03
  4.47400000e+03 1.18044755e+04]
 ...
 [4.00000000e+00 4.57600000e+03 1.00000000e+00 ... 3.83600000e+03
  1.11600000e+03 1.54860000e+04]
 [4.00000000e+00 3.69600000e+03 1.00000000e+00 ... 1.87200000e+03
  3.23000000e+02 1.40020000e+04]
 [4.00000000e+00 6.03000000e+03 1.00000000e+00 ... 4.76900000e+03
  4.87400000e+03 4.97200000e+03]]
[[0.00000000e+00 1.32500000e+03 1.00000000e+00 ... 1.29900000e+03
  1.68600000e+03 1.64410000e+04]
 [0.00000000e+00 1.72200000e+03 1.00000000e+00 ... 2.64100000e+03
  2.62900000e+03 1.03690000e+04]
 [0.00000000e+00 1.52400000e+03 1.00000000e+00 ... 2.34000000e+03
  4.92900000e+03 8.90200000e+03]
 ...
 [4.00000000e+00 2.89200000e+03 1.00000000e+00 ... 4.26900000e+03
  3.45218790e+03 2.00980000e+04]

In [13]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F

In [14]:
batch_size=100
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
train_ds = TensorDataset(torch.FloatTensor(train_X), torch.from_numpy(train_y))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
val_ds = TensorDataset(torch.FloatTensor(val_X), torch.from_numpy(val_y))
val_dl = DataLoader(val_ds, shuffle=True, batch_size=batch_size, drop_last=True)

In [16]:
test_ds = TensorDataset(torch.FloatTensor(test))
test_dl = DataLoader(test_ds, shuffle=False, batch_size=batch_size)
print(test_dl)

In [17]:

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(24, 48)
        self.layer_2 = nn.Linear(48, 48)
        self.layer_out = nn.Linear(48, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(24)
        self.batchnorm2 = nn.BatchNorm1d(48)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.batchnorm1(x)
        x = self.relu(self.layer_1(x))
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        #x = self.batchnorm2(x)
        #x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        #print(x.size())
        return x

In [18]:

model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

best_acc = 0
for epoch in range(20):
    print(f"Epoch [{epoch}]")
    model.train()
    for attr, label in train_dl:
        attr = attr.to(device)
        label = label.to(device,dtype=torch.float)
        #label = torch.cuda.LongTensor(label)
        
        pred = model(attr)
        #print(pred)
        #print(label)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    sample_count = 0
    correct_count = 0
    model.eval()
    for attr, label in val_dl:
        attr = attr.to(device)
        label = label.to(device,dtype=torch.float)
        
        pred = model(attr)
        #print(pred.size())
        #print(ohlabel.size())
        loss = loss_fn(pred, label)
        pred = torch.round(pred)
  
        #print(pred)
        sample_count += len(attr)
        correct_count += torch.eq(pred,label).int().sum()
    val_acc = correct_count / sample_count
    print("accuracy (validation):", val_acc)
    if best_acc <= val_acc:
        best_acc = val_acc
        PATH=f"model.pt"
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, PATH)



Epoch [0]
accuracy (validation): tensor(0.6509, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.6572, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.6492, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.6574, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.6458, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(0.6440, device='cuda:0')
Epoch [6]
accuracy (validation): tensor(0.6491, device='cuda:0')
Epoch [7]
accuracy (validation): tensor(0.6626, device='cuda:0')
Epoch [8]
accuracy (validation): tensor(0.6564, device='cuda:0')
Epoch [9]
accuracy (validation): tensor(0.6511, device='cuda:0')
Epoch [10]
accuracy (validation): tensor(0.6445, device='cuda:0')
Epoch [11]
accuracy (validation): tensor(0.6419, device='cuda:0')
Epoch [12]
accuracy (validation): tensor(0.6506, device='cuda:0')
Epoch [13]
accuracy (validation): tensor(0.6447, device='cuda:0')
Epoch [14]
accuracy (validation): tensor(0.6509, device='cuda:0')
Epoch [15]
accuracy 

In [19]:


#load model
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
print(epoch)
model.eval()
predict=[]
for attr in test_dl:
  #print(type(attr[0]))
  att = attr[0].to(device)
  
  pred = model(att)
  #pred = torch.gt(pred,0).int()
  #pred = torch.mul(pred, 0.5)
  #pred = torch.add(pred, 0.5)
  print(pred)
  for i in pred:
    predict.append(i.item())

8
tensor([[0.5151],
        [0.5194],
        [0.5095],
        [0.4851],
        [0.5469],
        [0.5208],
        [0.4539],
        [0.5110],
        [0.4837],
        [0.5026],
        [0.5010],
        [0.5376],
        [0.4982],
        [0.4782],
        [0.5108],
        [0.5335],
        [0.5501],
        [0.5261],
        [0.5270],
        [0.4793],
        [0.5462],
        [0.5123],
        [0.5328],
        [0.5152],
        [0.5172],
        [0.5180],
        [0.4984],
        [0.4933],
        [0.5136],
        [0.5141],
        [0.4815],
        [0.5297],
        [0.5177],
        [0.4678],
        [0.4804],
        [0.5506],
        [0.4651],
        [0.5186],
        [0.4705],
        [0.5305],
        [0.4961],
        [0.4944],
        [0.5058],
        [0.4851],
        [0.5175],
        [0.5357],
        [0.4904],
        [0.5398],
        [0.4928],
        [0.4768],
        [0.4885],
        [0.5128],
        [0.4857],
        [0.5576],
        [0.5142],
        

In [20]:
print(len(predict))
print(len(ids))

20775
20775


In [21]:
import csv
with open('submission.csv', 'w', newline='') as f:
  csv_writer = csv.writer(f)
  csv_writer.writerow(["id", "failure"])
  count = 0
  for id, p in zip(ids,predict):
    csv_writer.writerow([id,p])

In [22]:
print(count)

0
